## Нейронная маркировка части речи

Мы не собираемся решать ту же проблему с маркировкой POS с помощью нейронных сетей.
<img src=https://i.stack.imgur.com/6pdIT.png width=320>

С точки зрения глубокого обучения, это задача прогнозирования последовательности выходных данных, согласованной с последовательностью входных данных. Есть несколько задач, которые соответствуют этой формулировке:
* * Выделение тегов части речи - вспомогательная задача для многих задач NLP
* Распознавание именованных объектов - для чат-ботов и веб-сканеров
* Прогнозирование структуры белка - для биоинформатики

In [71]:
import nltk
import sys
import numpy as np

nltk.download('brown')
nltk.download('universal_tagset')
data = nltk.corpus.brown.tagged_sents(tagset='universal')
all_tags = ['#EOS#','#UNK#','ADV', 'NOUN', 'ADP', 'PRON', 'DET', '.', 'PRT', 'VERB', 'X', 'NUM', 'CONJ', 'ADJ']

data = np.array([[(word.lower(), tag) for word, tag in sentence] for sentence in data], dtype=object)

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\avsip\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\avsip\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [72]:
# тестирование модели
def predict_tags(model, word_to_id, test_sentence):
    test_sequence = [word_to_id.get(word.lower(), 0) for word in test_sentence]
    predicted_tags_probabilities = model.predict(np.array([test_sequence]))
    predicted_tags = np.argmax(predicted_tags_probabilities, axis=-1)[0]
    predicted_tags_text = [all_tags[tag_id] for tag_id in predicted_tags]
    return predicted_tags_text

In [73]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data,test_size=0.25,random_state=42)

In [74]:
from IPython.display import HTML, display
def draw(sentence):
    words,tags = zip(*sentence)
    display(HTML('<table><tr>{tags}</tr>{words}<tr></table>'.format(
                words = '<td>{}</td>'.format('</td><td>'.join(words)),
                tags = '<td>{}</td>'.format('</td><td>'.join(tags)))))

draw(data[11])
draw(data[10])
draw(data[7])

NOUN,ADP,NOUN,NOUN,NOUN,NOUN,VERB,ADV,VERB,ADP,DET,ADJ,NOUN,.,implementation,of,georgia's,automobile,title,law,was,also,recommended,by,the,outgoing,jury,.


PRON,VERB,ADP,DET,NOUN,.,VERB,NOUN,PRT,VERB,.,DET,NOUN,.,it,urged,that,the,city,``,take,steps,to,remedy,'',this,problem,.


NOUN,VERB,merger,proposed


### Построение словарей

Как и раньше, мы должны построить отображение от токенов к целочисленным идентификаторам. На этот раз наша модель работает на уровне слов, обрабатывая одно слово за шаг RNN. Это означает, что нам придется иметь дело с гораздо большим словарным запасом.
К счастью для нас, мы получаем только эти слова в качестве входных данных, т.е. нам не нужно их предсказывать. Это означает, что мы можем бесплатно получить большой словарный запас, используя встраивания слов.

In [75]:
from collections import Counter
word_counts = Counter()
for sentence in data:
    words,tags = zip(*sentence)
    word_counts.update(words)

all_words = ['#EOS#','#UNK#'] + list(list(zip(*word_counts.most_common(10000)))[0])

print("Coverage = %.5f" % (float(sum(word_counts[w] for w in all_words)) / sum(word_counts.values())))

Coverage = 0.92876


In [76]:
from collections import defaultdict
word_to_id = defaultdict(lambda:1, { word: i for i, word in enumerate(all_words) })
tag_to_id = { tag: i for i, tag in enumerate(all_tags)}

преобразуйте слова и теги в матрицу фиксированного размера

In [77]:
def to_matrix(lines, token_to_id, max_len=None, pad=0, dtype='int32', time_major=False):
    """Converts a list of names into rnn-digestable matrix with paddings added after the end"""

    max_len = max_len or max(map(len,lines))
    matrix = np.empty([len(lines), max_len],dtype)
    matrix.fill(pad)

    for i in range(len(lines)):
        line_ix = list(map(token_to_id.__getitem__,lines[i]))[:max_len]
        matrix[i,:len(line_ix)] = line_ix

    return matrix.T if time_major else matrix

In [78]:
batch_words, batch_tags = zip(*[zip(*sentence) for sentence in data[-3:]])

print("Word ids:")
print(to_matrix(batch_words, word_to_id))
print("Tag ids:")
print(to_matrix(batch_tags, tag_to_id))

Word ids:
[[   2 3057    5    2 2238 1334 4238 2454    3    6   19   26 1070   69
     8 2088    6    3    1    3  266   65  342    2    1    3    2  315
     1    9   87  216 3322   69 1558    4    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  45   12    8  511 8419    6   60 3246   39    2    1    1    3    2
   845    1    3    1    3   10 9910    2    1 3470    9   43    1    1
     3    6    2 1046  385   73 4562    3    9    2    1    1 3250    3
    12   10    2  861 5240   12    8 8936  121    1    4]
 [  33   64   26   12  445    7 7346    9    8 3337    3    1 2811    3
     2  463  572    2    1    1 1649   12    1    4    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
Tag ids:
[[ 6  3  4  6  3  3  9  9  7 12  4  5  9  4  6  3 12  7  9  7  9  8  4  6
   3  7  6 13  3  4  6  3  9  4  3  7  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0

### Построить модель

В отличие от нашей предыдущей лабораторной работы, на этот раз мы сосредоточимся на высокоуровневом интерфейсе keras для рекуррентных нейронных сетей.
Он настолько прост, насколько это возможно с помощью RUN, хотя и несколько ограничен для сложных задач, таких как seq2 seq.
По умолчанию все RNN keras применяются ко всей последовательности входных данных и выдают последовательность скрытых состояний `(return_sequences=True`
или только последнее скрытое состояние `(return_sequences=False)`. Все повторение происходит под капотом.
В верхней части нашей модели нам нужно нанести плотный слой на каждый временной шаг независимо.
На данный момент по умолчанию используется keras.слои.Dense будет применяться один раз ко всем объединенным временным шагам.
Мы используем __keras.слои.TimeDistributed__ для изменения плотного слоя таким образом, чтобы он применялся как по пакетной, так и по временной осям.

In [79]:
import keras
import keras.layers as L
import tensorflow as tf

model = keras.models.Sequential()
model.add(L.InputLayer([None],dtype='int32'))
model.add(L.Embedding(len(all_words),50))
model.add(L.SimpleRNN(64,return_sequences=True))
stepwise_dense = L.Dense(len(all_tags),activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

# исправление Warning ()
# tf_loss_function = tf.compat.v1.losses.sparse_softmax_cross_entropy
# tf_default_graph = tf.compat.v1.get_default_graph

__ Обучение:__ в этом случае мы не хотим заранее подготавливать весь набор обучающих данных. Основная причина заключается в том, что длина каждого пакета зависит от максимальной длины предложения в пакете. Это оставляет нам два варианта: использовать пользовательский обучающий код, как на предыдущем семинаре, или использовать генераторы.

Модели Keras имеют метод __`model.fit_generator`__, который принимает генератор python, выдающий по одному пакету за раз. Но сначала нам нужно реализовать такой генератор:

In [80]:
import tensorflow
from tensorflow.keras.utils import to_categorical

BATCH_SIZE=32
def generate_batches(sentences,batch_size=BATCH_SIZE,max_len=None,pad=0):
    assert isinstance(sentences,np.ndarray),"Make sure sentences is q numpy array"

    while True:
        indices = np.random.permutation(np.arange(len(sentences)))
        for start in range(0,len(indices)-1,batch_size):
            batch_indices = indices[start:start+batch_size]
            batch_words,batch_tags = [],[]
            for sent in sentences[batch_indices]:
                words,tags = zip(*sent)
                batch_words.append(words)
                batch_tags.append(tags)

            batch_words = to_matrix(batch_words,word_to_id,max_len,pad)
            batch_tags = to_matrix(batch_tags,tag_to_id,max_len,pad)

            batch_tags_1hot = to_categorical(batch_tags,len(all_tags)).reshape(batch_tags.shape+(-1,))
            yield batch_words,batch_tags_1hot


__Callbacks:__ Еще одна вещь, которая нам нужна, - это измерение производительности модели. Сложность заключается в том, чтобы не подсчитывать точность после окончания предложения (при заполнении) и убедиться, что мы подсчитываем все данные проверки ровно один раз.

Хотя убедить Keras сделать все это не невозможно, мы также можем написать наш собственный обратный вызов, который это делает.
Обратные вызовы Keras позволяют вам писать пользовательский код, который будет запускаться один раз в каждую эпоху или в каждом мини-пакете. Мы определим его с помощью обратного вызова Lambda

In [81]:
def compute_test_accuracy(model):
    test_words,test_tags = zip(*[zip(*sentence) for sentence in test_data])
    test_words,test_tags = to_matrix(test_words,word_to_id),to_matrix(test_tags,tag_to_id)

    #predict tag probabilities of shape [batch,time,n_tags]
    predicted_tag_probabilities = model.predict(test_words,verbose=1)
    predicted_tags = predicted_tag_probabilities.argmax(axis=-1)

    #compute accurary excluding padding
    numerator = np.sum(np.logical_and((predicted_tags == test_tags),(test_words != 0)))
    denominator = np.sum(test_words != 0)
    return float(numerator)/denominator


class EvaluateAccuracy(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        sys.stdout.flush()
        print("\nMeasuring validation accuracy...")
        acc = compute_test_accuracy(self.model)
        print("\nValidation accuracy: %.5f\n"%acc)
        sys.stdout.flush()

In [82]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5


C:\Users\avsip\AppData\Local\Temp\ipykernel_15672\1670692242.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,


1344/1343 [==============================] - ETA: 0s - loss: 0.2433
Measuring validation accuracy...
448/448 [==============================] - 5s 10ms/step

Validation accuracy: 0.94084

1343/1343 [==============================] - 26s 18ms/step - loss: 0.2433
Epoch 2/5
1342/1343 [============================>.] - ETA: 0s - loss: 0.0578
Measuring validation accuracy...
448/448 [==============================] - 5s 10ms/step

Validation accuracy: 0.94508

1343/1343 [==============================] - 23s 17ms/step - loss: 0.0578
Epoch 3/5
1342/1343 [============================>.] - ETA: 0s - loss: 0.0511
Measuring validation accuracy...
448/448 [==============================] - 5s 10ms/step

Validation accuracy: 0.94680

1343/1343 [==============================] - 25s 18ms/step - loss: 0.0511
Epoch 4/5
1342/1343 [============================>.] - ETA: 0s - loss: 0.0469
Measuring validation accuracy...
448/448 [==============================] - 5s 11ms/step

Validation accuracy: 0.945

In [83]:
acc = compute_test_accuracy(model)
print("Final accuracy: %.5f"%acc)

assert acc>0.94, "Keras has gone on a rampage again, please contact course staff."

448/448 [==============================] - 5s 11ms/step
Final accuracy: 0.94618


In [84]:
# Пример использования
test_sentence = ["This", "is", "a", "test", "sentence"]
predicted_tags_result = predict_tags(model, word_to_id, test_sentence)

print("Input Sentence:", test_sentence)
print("Predicted Tags:", predicted_tags_result)

1/1 [==============================] - 0s 138ms/step
Input Sentence: ['This', 'is', 'a', 'test', 'sentence']
Predicted Tags: ['DET', 'VERB', 'DET', 'NOUN', 'NOUN']


### Двунаправленный анализ

Поскольку мы анализируем полную последовательность, для нас законно просматривать будущие данные.
Простой способ добиться этого - работать в обоих направлениях одновременно, создавая __двунаправленный RNN__.
В Keras вы можете добиться этого как вручную (используя два элемента и объединение), так и с помощью __`keras.слои.Bidirectional`__.
Это работает так же, как "Распределенное по времени", которое мы видели ранее: вы оборачиваете его вокруг повторяющегося слоя (SimpleRNN сейчас и LSTM / GRU позже), и это фактически создает два слоя под капотом.
Ваша первая задача - использовать такой слой в нашем POS-теггере.

In [85]:
model = keras.models.Sequential()
model.add(L.InputLayer([None], dtype='int32'))
model.add(L.Embedding(len(all_words), 50))
model.add(L.Bidirectional(L.SimpleRNN(64, return_sequences=True)))
stepwise_dense = L.Dense(len(all_tags), activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

In [86]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5


C:\Users\avsip\AppData\Local\Temp\ipykernel_15672\1670692242.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,


1344/1343 [==============================] - ETA: 0s - loss: 0.2044
Measuring validation accuracy...
448/448 [==============================] - 9s 20ms/step

Validation accuracy: 0.95564

1343/1343 [==============================] - 39s 28ms/step - loss: 0.2044
Epoch 2/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0430
Measuring validation accuracy...
448/448 [==============================] - 10s 22ms/step

Validation accuracy: 0.96092

1343/1343 [==============================] - 41s 31ms/step - loss: 0.0430
Epoch 3/5
1343/1343 [============================>.] - ETA: 0s - loss: 0.0352
Measuring validation accuracy...
448/448 [==============================] - 10s 22ms/step

Validation accuracy: 0.96267

1343/1343 [==============================] - 40s 30ms/step - loss: 0.0352
Epoch 4/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0297
Measuring validation accuracy...
448/448 [==============================] - 10s 22ms/step

Validation accuracy: 0.

In [87]:
acc = compute_test_accuracy(model)
print("\nFinal accuracy: %.5f"%acc)

assert acc>0.96, "Bidirectional RNNs are better than this!"
print("Well done!")

448/448 [==============================] - 9s 21ms/step

Final accuracy: 0.96187
Well done!


In [88]:
# Пример использования
test_sentence = ["This", "is", "a", "test", "sentence"]
predicted_tags_result = predict_tags(model, word_to_id, test_sentence)

print("Input Sentence:", test_sentence)
print("Predicted Tags:", predicted_tags_result)

1/1 [==============================] - 0s 249ms/step
Input Sentence: ['This', 'is', 'a', 'test', 'sentence']
Predicted Tags: ['DET', 'VERB', 'DET', 'ADJ', 'NOUN']


Создайте **хотя бы один эксперимент** из приведенного ниже списка вы можете выбрать наиболее интересные и многообещающие варианты повышения производительности двунаправленного LSTM:

* __Go beyond SimpleRNN__: there's `keras.layers.LSTM` and `keras.layers.GRU`
  * Если вы хотите использовать пользовательскую рекуррентную ячейку, прочитайте [это](https://keras.io/layers/recurrent/#rnn)
  * Вы также можете использовать одномерные свертки (`keras.слои.Conv1D`). Они часто так же хороши, как и повторяющиеся слои, но требуют меньшего переобучения.
* __Stack more layers__: если в этом курсе и есть общий мотив, то он касается укладки слоев
  * Вы можете просто добавить слои recurrent и `1dconv` один поверх другого, и `keras` это поймет
  * Просто помните, что большим сетям может потребоваться больше эпох для обучения
* __Regularization__: вы можете применять отсевы как обычно, но также и специфичным для RNN способом
  * `keras.laers.Dropout` работает между слоями RNN
  * Повторяющиеся слои также имеют параметр `recurrent_dropout`
*  __Gradient clipping__: Если ваше обучение не так стабильно, как вам хотелось бы, установите `clipnorm` в вашем оптимизаторе.
   * То есть, это хорошая идея - следить за своей кривой потерь при каждом минибатче. Попробуйте обратный вызов `tensorboard` или что-то подобное.

__Go beyond SimpleRNN__ add `L.Bidirectional(L.LSTM)`

In [89]:
model = keras.models.Sequential()
model.add(L.InputLayer([None], dtype='int32'))
model.add(L.Embedding(len(all_words), 50))

# Bidirectional LSTM
model.add(L.Bidirectional(L.LSTM(64, return_sequences=True)))
stepwise_dense = L.Dense(len(all_tags), activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

model.compile('adam','categorical_crossentropy')
model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5


C:\Users\avsip\AppData\Local\Temp\ipykernel_15672\3956650167.py:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,


1344/1343 [==============================] - ETA: 0s - loss: 0.2474
Measuring validation accuracy...
448/448 [==============================] - 25s 54ms/step

Validation accuracy: 0.95396

1343/1343 [==============================] - 101s 72ms/step - loss: 0.2474
Epoch 2/5
1343/1343 [============================>.] - ETA: 0s - loss: 0.0456
Measuring validation accuracy...
448/448 [==============================] - 23s 51ms/step

Validation accuracy: 0.96043

1343/1343 [==============================] - 93s 69ms/step - loss: 0.0456
Epoch 3/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0372
Measuring validation accuracy...
448/448 [==============================] - 22s 50ms/step

Validation accuracy: 0.96315

1343/1343 [==============================] - 90s 67ms/step - loss: 0.0372
Epoch 4/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0326
Measuring validation accuracy...
448/448 [==============================] - 23s 52ms/step

Validation accuracy: 

In [90]:
acc = compute_test_accuracy(model)
print("\nFinal accuracy Bidirectional LSTM: %.5f"%acc)


448/448 [==============================] - 23s 51ms/step

Final accuracy Bidirectional LSTM: 0.96493


In [91]:
# Пример использования
test_sentence = ["This", "is", "a", "test", "sentence"]
predicted_tags_result = predict_tags(model, word_to_id, test_sentence)

print("Input Sentence:", test_sentence)
print("Predicted Tags:", predicted_tags_result)

1/1 [==============================] - 1s 591ms/step
Input Sentence: ['This', 'is', 'a', 'test', 'sentence']
Predicted Tags: ['DET', 'VERB', 'DET', 'NOUN', 'NOUN']


__Stack more layers__

In [92]:
model = keras.models.Sequential()
model.add(L.InputLayer([None], dtype='int32'))
model.add(L.Embedding(len(all_words), 50))
model.add(L.Bidirectional(L.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(L.Bidirectional(L.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(L.Conv1D(64, kernel_size=3, padding='same', activation='relu'))
stepwise_dense = L.Dense(len(all_tags), activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

model.compile('adam','categorical_crossentropy')
model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5


C:\Users\avsip\AppData\Local\Temp\ipykernel_15672\2297888989.py:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,


1344/1343 [==============================] - ETA: 0s - loss: 0.2625
Measuring validation accuracy...
448/448 [==============================] - 27s 59ms/step

Validation accuracy: 0.94954

1343/1343 [==============================] - 200s 141ms/step - loss: 0.2625
Epoch 2/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0602
Measuring validation accuracy...
448/448 [==============================] - 25s 56ms/step

Validation accuracy: 0.95637

1343/1343 [==============================] - 175s 130ms/step - loss: 0.0602
Epoch 3/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0498
Measuring validation accuracy...
448/448 [==============================] - 18s 41ms/step

Validation accuracy: 0.96045

1343/1343 [==============================] - 170s 126ms/step - loss: 0.0498
Epoch 4/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0436
Measuring validation accuracy...
448/448 [==============================] - 17s 38ms/step

Validation accur

In [93]:
model.compile('adam', 'categorical_crossentropy')
acc = compute_test_accuracy(model)
print("\nFinal accuracy Bidirectional LSTM and Bidirectional LSTM: %.5f"%acc)

448/448 [==============================] - 18s 38ms/step

Final accuracy Bidirectional LSTM and Bidirectional LSTM: 0.96381


In [94]:
# Пример использования
test_sentence = ["This", "is", "a", "test", "sentence"]
predicted_tags_result = predict_tags(model, word_to_id, test_sentence)

print("Input Sentence:", test_sentence)
print("Predicted Tags:", predicted_tags_result)

1/1 [==============================] - 1s 551ms/step
Input Sentence: ['This', 'is', 'a', 'test', 'sentence']
Predicted Tags: ['DET', 'VERB', 'DET', 'NOUN', 'NOUN']


__Regularization__ `Dropout`

In [95]:
model = keras.models.Sequential()
model.add(L.InputLayer([None], dtype='int32'))
model.add(L.Embedding(len(all_words), 50))
model.add(L.Bidirectional(L.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(L.Bidirectional(L.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(L.Dropout(0.5))
model.add(L.Conv1D(64, kernel_size=3, padding='same', activation='relu'))
stepwise_dense = L.Dense(len(all_tags), activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

model.compile('adam','categorical_crossentropy')
model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5


C:\Users\avsip\AppData\Local\Temp\ipykernel_15672\1620620932.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,


1344/1343 [==============================] - ETA: 0s - loss: 0.3130
Measuring validation accuracy...
448/448 [==============================] - 18s 38ms/step

Validation accuracy: 0.94708

1343/1343 [==============================] - 153s 108ms/step - loss: 0.3130
Epoch 2/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0663
Measuring validation accuracy...
448/448 [==============================] - 17s 39ms/step

Validation accuracy: 0.95597

1343/1343 [==============================] - 148s 110ms/step - loss: 0.0663
Epoch 3/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0536
Measuring validation accuracy...
448/448 [==============================] - 17s 38ms/step

Validation accuracy: 0.95948

1343/1343 [==============================] - 148s 110ms/step - loss: 0.0536
Epoch 4/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0473
Measuring validation accuracy...
448/448 [==============================] - 17s 38ms/step

Validation accur

In [96]:
model.compile('adam', 'categorical_crossentropy')
acc = compute_test_accuracy(model)
print("\nFinal accuracy Regularization (Dropout): %.5f"%acc)

448/448 [==============================] - 18s 39ms/step

Final accuracy Regularization (Dropout): 0.96374


In [97]:
# Пример использования
test_sentence = ["This", "is", "a", "test", "sentence"]
predicted_tags_result = predict_tags(model, word_to_id, test_sentence)

print("Input Sentence:", test_sentence)
print("Predicted Tags:", predicted_tags_result)

1/1 [==============================] - 1s 569ms/step
Input Sentence: ['This', 'is', 'a', 'test', 'sentence']
Predicted Tags: ['DET', 'VERB', 'DET', 'NOUN', 'NOUN']


__Gradient clipping__

In [98]:
model = keras.models.Sequential()
model.add(L.InputLayer([None], dtype='int32'))
model.add(L.Embedding(len(all_words), 50))
model.add(L.Bidirectional(L.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(L.Bidirectional(L.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(L.Dropout(0.5))
model.add(L.Conv1D(64, kernel_size=3, padding='same', activation='relu'))
stepwise_dense = L.Dense(len(all_tags), activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

model.compile(optimizer=keras.optimizers.Adam(clipnorm=1.0), loss='categorical_crossentropy')
model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5


C:\Users\avsip\AppData\Local\Temp\ipykernel_15672\1404121170.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,


1344/1343 [==============================] - ETA: 0s - loss: 0.3126
Measuring validation accuracy...
448/448 [==============================] - 18s 38ms/step

Validation accuracy: 0.94626

1343/1343 [==============================] - 156s 111ms/step - loss: 0.3126
Epoch 2/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0691
Measuring validation accuracy...
448/448 [==============================] - 17s 38ms/step

Validation accuracy: 0.95516

1343/1343 [==============================] - 149s 111ms/step - loss: 0.0691
Epoch 3/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0546
Measuring validation accuracy...
448/448 [==============================] - 18s 39ms/step

Validation accuracy: 0.95796

1343/1343 [==============================] - 153s 114ms/step - loss: 0.0546
Epoch 4/5
1344/1343 [==============================] - ETA: 0s - loss: 0.0480
Measuring validation accuracy...
448/448 [==============================] - 18s 39ms/step

Validation accur

In [99]:
model.compile('adam', 'categorical_crossentropy')
acc = compute_test_accuracy(model)
print("\nFinal accuracy Regularization (Dropout) and clipnorm: %.5f"%acc)

448/448 [==============================] - 18s 39ms/step

Final accuracy Regularization (Dropout) and clipnorm: 0.96343


In [100]:
# Пример использования
test_sentence = ["This", "is", "a", "test", "sentence"]
predicted_tags_result = predict_tags(model, word_to_id, test_sentence)

print("Input Sentence:", test_sentence)
print("Predicted Tags:", predicted_tags_result)

1/1 [==============================] - 1s 548ms/step
Input Sentence: ['This', 'is', 'a', 'test', 'sentence']
Predicted Tags: ['DET', 'VERB', 'DET', 'NOUN', 'NOUN']


### Вывод

`SimpleRNN`:
 + точность: `94.618%`
 + Предсказанные теги: `['DET', 'VERB', 'DET', 'NOUN', 'NOUN']`

`Bidirectional SimpleRNN`:
 + точность: `96.187%`
 + Предсказанные теги: `['DET', 'VERB', 'DET', 'ADJ', 'NOUN']`
 + Введение двунаправленности улучшило точность до `96.187%`.
  
`Bidirectional LSTM:` &#9989;
 + точность: `96.493%`
 + Предсказанные теги: `['DET', 'VERB', 'DET', 'NOUN', 'NOUN']`
 + Использование `Bidirectional LSTM` дополнительно увеличило точность до `96.493%`.
 + Предсказанные теги, остались схожими с `Bidirectional SimpleRNN`.

Двойной `Bidirectional LSTM`:
 + точность: `96.381%`
 + Предсказанные теги: `['DET', 'VERB', 'DET', 'NOUN', 'NOUN']`
 + Использование двух слоев `Bidirectional LSTM` снизило точность `96.381%`.

`Bidirectional LSTM с Dropout (0.5)`:
 + точность: `96.374%`
 + Предсказанные теги: `['DET', 'VERB', 'DET', 'NOUN', 'NOUN']`
 + `dropout` `0.5` не существенно изменило точность (`96.374%`)

`Bidirectional LSTM с Dropout (0.5) и clipnorm (1.0):`
 + точность: `96.343%`
 + Предсказанные теги: `['DET', 'VERB', 'DET', 'NOUN', 'NOUN']`
 + Применение как `dropout`, так и `clipnorm` в оптимизаторе снизило точность до `96.343%`

`Bidirectional LSTM` &#9989; лучший выбором для данной задачи, поскольку он превосходит `SimpleRNN`.